input space:
 - 2d closed interval of real numbers, `[-1,1]x[-1,1]`, with a uniform probability across the intervals of picking an _`x`_


target function _`f`_: 
 - random line in the plane
 - defined by two random points
 - one side of the line maps to +1, the other to -1
 - choose random inputs xn of the data set, and evaluate each to get the corresponding output yn



In [2]:
import numpy as np


In [3]:
def choose_points(n, seed):
    rng = np.random.default_rng(seed=seed)
    return (rng.random((n,2)) * 2) - 1


def choose_line(seed):
    rng = np.random.default_rng(seed=seed)
    r = (rng.random(4) * 2) - 1
    l = {'x1': r[0], 'y1': r[1], 'x2': r[2], 'y2': r[3]}
    print('Points')
    print(l)
    m = (l['y2'] - l['y1']) / (l['x2'] - l['x1'])
    b = l['y1'] - (m * l['x1'])
    line = {"m": m, "b": b}
    print('Eq')
    print(line)
    return(line)


def assign_y(point, target_fxn):
    """ 
    Determine whether the point is +1 or -1, 
    depending on whether it's to the right or left of the line
    """
    is_right = ((target_fxn['m'] * point[0]) + target_fxn['b']) < point[1]
    if is_right:
        return 1
    else:
        return -1


def make_target_function(target_fxn):
    """
    should return a function that takes only a point
    """
    def assign_y(point):
        is_right = ((target_fxn['m'] * point[0]) + target_fxn['b']) < point[1]
        if is_right:
            return 1
        else:
            return -1
    return assign_y


In [38]:
def initialize_data(seed, point_count):
    points = choose_points(point_count, seed=seed)
    target_fxn = choose_line(seed=seed+1)
    fxn = make_target_function(target_fxn)
    Y_output = np.apply_along_axis(fxn, 1, points)
    return points, Y_output, target_fxn



seed = 41
point_count = 10

points, Y_output, target_fxn = initialize_data(seed=seed, point_count=point_count)

print("Y")
print(Y_output)
print(points)

Points
{'x1': 0.5479120971119267, 'y1': -0.12224312049589536, 'x2': 0.7171958398227649, 'y2': 0.3947360581187278}
Eq
{'m': 3.053921010582217, 'b': -1.7955233858181725}
Y
[-1  1 -1  1  1  1  1  1 -1 -1]
[[ 0.90830221  0.53586418]
 [-0.74805851  0.65397617]
 [ 0.69814415 -0.30043433]
 [ 0.02281847  0.20499982]
 [-0.32232817  0.2436128 ]
 [ 0.46496935  0.72217656]
 [-0.32435121 -0.14142151]
 [ 0.18836268  0.95135587]
 [ 0.85573835 -0.88303505]
 [ 0.53004364 -0.6849098 ]]


In [90]:
def classification(weight_vector, point):
    """ the classification of a point given the weight vector and the point """
    # add a dummy variable to the point vector, which should be the same length as the weight vector
    x_vector = np.array([1] + list(point))
    value = np.inner(weight_vector, x_vector)
    return np.sign(value)


def update_weight_vector(weight_vector, y, point):
    update_vector = np.array([1] + list(point)) * y
    return weight_vector + update_vector


def iteration(weight_vector, points, target_function):
    """
    Check each point in the vector of points until you find one that does not agree.
    Then, update the weight vector.
    """
    new_weight_vector = weight_vector
    for point in points:
        updated = False
        result = classification(weight_vector, point)
        y = assign_y(point, target_function)
        if result != y:
            # print(weight_vector, point, y, result)
            new_weight_vector = update_weight_vector(weight_vector, y, point)
            # print(f"updating weight vector to: {new_weight_vector}")
            updated = True
            break
    return new_weight_vector, updated

# for the perceptron learning algorithm, we need a weight vector, w
# starting filled with zeros

weight_vector = np.zeros(3)
required_update = True
iteration_counter = 0
points = np.random.permutation(points)
while required_update:
    iteration_counter += 1
    weight_vector, required_update = iteration(weight_vector, points, target_fxn)

print(f"iterations: {iteration_counter}")
print(f"final weight vector: {weight_vector}")

iterations: 8
final weight vector: [ 1.         -1.65232675  0.80254994]


In [ ]:
# We are interested in two quantities: 
# the number of iterations that PLA takes to converge to g, 
# and 
# the disagreement between f and g which is P[f(x) 6= g(x)] 
# (the probability that f and g will disagree on their classification of a random point).
# You can either calculate this probability exactly, or
# approximate it by generating a sufficiently large, separate set of points to estimate it.

In [ ]:
# generate data for disagreement
# how many points? 10k, assuming about 100x100 density on each side; 
# you can then calculate the probability by dividing the number of points in the diff between f and g divided by 10k

In [18]:
np.random.permutation(np.array([[1,0],[2,0],[3,0],[4,0],[5,0]]))

array([[3, 0],
       [2, 0],
       [5, 0],
       [4, 0],
       [1, 0]])

In [128]:
def run(n, seed):
    """ 
    run through an experiment to get: 
     - an average number of iterations to converge
     - probability that f and g will disagree on their classification of a random point
     
    Note:
     - the points and the initial line that generates the truth set is determined by the seed
    """
    run_iterations = 1000
    point_count_for_prob_calc = 10000
    required_update = True
    # initialize data
    points, Y_output, target_fxn = initialize_data(seed=seed, point_count=n)
    print(Y_output)
    iteration_counts = []
    disagree_probs = []
    for i in range(run_iterations):
        weight_vector = np.zeros(3)
        iteration_counter = 0
        points = np.random.permutation(points)
        required_update = True
        while required_update:
            iteration_counter += 1
            weight_vector, required_update = iteration(weight_vector, points, target_fxn)
        iteration_counts.append(iteration_counter)
    iteration_count_mean = np.array(iteration_counts).mean()
    
    prob_counter = {"agree": 0, "disagree": 0}
    for prob_calc_point in choose_points(point_count_for_prob_calc, seed + 1):
        if assign_y(prob_calc_point, target_fxn) == classification(weight_vector, prob_calc_point):
            prob_counter["agree"] += 1
        else:
            prob_counter["disagree"] += 1
    prob_disagree = (prob_counter["disagree"] / (prob_counter["disagree"] + prob_counter["agree"]))
    return iteration_count_mean, prob_disagree

for x in range(10):
    print(run(100, 100 + x))

Points
{'x1': 0.8870650112211078, 'y1': -0.2811579333168537, 'x2': 0.5696108239399542, 'y2': 0.18255637045882356}
Eq
{'m': -1.460728263650176, 'b': 1.014603000268979}
[ 1 -1  1  1 -1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1  1  1
  1 -1 -1 -1 -1 -1  1  1 -1  1 -1  1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
 -1  1 -1  1 -1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
(218.273, 0.0085)
Points
{'x1': -0.6800177099913927, 'y1': 0.17188702968045466, 'x2': 0.6476735543656926, 'y2': -0.40547952647940755}
Eq
{'m': -0.43486507116505235, 'b': -0.12382892016844826}
[ 1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1  1  1  1  1 -1  1  1 -1 -1 -1  1
 -1  1  1  1 -1 -1 -1 -1  1  1  1  1  1  1  1  1  1 -1  1  1 -1  1 -1 -1
 -1  1  1 -1  1 -1 -1  1  1 -1  1 -1  1  1 -1  1 -1  1 -1  1  1  1  1 -1
 -1  1  1  1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1  1 -1  1  1  1 -1 -1  1  1
  1  1  1  1]
(1350.028, 0.0023)
Points
{'x

In [130]:
results = []
for x in range(10):
    results.append(run(10, 200 + x))

Points
{'x1': 0.9814674897583036, 'y1': 0.07326665573470836, 'x2': -0.023317302628337977, 'y2': 0.18394878682424665}
Eq
{'m': -0.11015506198759004, 'b': 0.18138026790783868}
[ 1 -1 -1 -1 -1 -1  1 -1 -1  1]
Points
{'x1': -0.3074250001525154, 'y1': -0.11268818405118197, 'x2': 0.631510267152902, 'y2': 0.37498336156697265}
Eq
{'m': 0.5193878242721546, 'b': 0.04698461790489977}
[-1  1 -1  1  1  1 -1 -1 -1  1]
Points
{'x1': 0.5618860988495322, 'y1': -0.3113839426342788, 'x2': -0.8251102552851475, 'y2': -0.5380415211230485}
Eq
{'m': 0.1634161314217563, 'b': -0.4032051952079319}
[1 1 1 1 1 1 1 1 1 1]
Points
{'x1': 0.07287018547836555, 'y1': 0.6239502159395316, 'x2': 0.2371907621681706, 'y2': -0.5506249384005117}
Eq
{'m': -7.148071032864855, 'b': 1.1448314779169255}
[ 1 -1 -1  1  1 -1  1 -1 -1  1]
Points
{'x1': 0.8163712301939274, 'y1': -0.5438794118676407, 'x2': 0.6461697525423291, 'y2': -0.6192186028331692}
Eq
{'m': 0.4426471027457678, 'b': -0.9052437716779809}
[1 1 1 1 1 1 1 1 1 1]
Points
{'

In [131]:
results

[(16.502, 0.0891),
 (10.924, 0.0275),
 (2.0, 0.2943),
 (5.834, 0.0782),
 (2.392, 0.0943),
 (3.856, 0.0706),
 (31.532, 0.0639),
 (8.471, 0.1099),
 (3.528, 0.1875),
 (14.464, 0.124)]